In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import tensorflow_datasets as tfds

In [2]:
ds = pd.read_csv("/kaggle/input/quotes-500k/quotes.csv")

ds.head()

ds = ds.drop(columns=["author", "category"], axis=1)

ds = np.array(ds)
ds = ds.T[0].astype(str)

ds = ds[0:10000]

ds = np.char.lower(ds)

ds = np.array(list(map(lambda x: re.sub("[^a-z0-9\s]+", "", x), ds)))

print(ds.shape)
print(ds[0:3])

(10000,)
['im selfish impatient and a little insecure i make mistakes i am out of control and at times hard to handle but if you cant handle me at my worst then you sure as hell dont deserve me at my best'
 'youve gotta dance like theres nobody watchinglove like youll never be hurtsing like theres nobody listeningand live like its heaven on earth'
 'you know youre in love when you cant fall asleep because reality is finally better than your dreams']


In [3]:
lens = np.array(list(map(len, ds)))

median_length = np.median(lens).astype(np.int32)
max_length = np.amax(lens).astype(np.int32)

print(median_length, max_length)

# ds[291945]

115 3786


In [4]:
max_tokens = 10000

vocab = sorted(set("".join(ds)))
vocab_size = len(vocab) + 1

string_lookup = tf.keras.layers.StringLookup(max_tokens=max_tokens, vocabulary=vocab, mask_token=None)
ids_lookup = tf.keras.layers.StringLookup(vocabulary=string_lookup.get_vocabulary(), invert=True, mask_token=None)

print(len(vocab))
print(vocab[0:10])

string = np.array(["h", "e", "l", "l", "o"])
ids = string_lookup(string)

chars = ids_lookup(ids)
print(ids.numpy())
print(chars.numpy())

40
['\t', ' ', '0', '1', '2', '3', '4', '5', '6', '7']
[20 17 24 24 27]
[b'h' b'e' b'l' b'l' b'o']


2022-12-23 15:43:55.036489: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
def tokenize(x):
    x = tf.strings.unicode_split(x, "UTF-8")
    x = string_lookup(x)
    return x.numpy()

def pad(x):
    return tf.keras.preprocessing.sequence.pad_sequences(x, padding="post", truncating="post", maxlen=median_length)

def split_input_sequence(x):
    input_text = x[:-1]
    target_text = x[1:]
    return input_text, target_text

dataset = np.array(list(map(tokenize, ds)))

dataset = pad(dataset)

dataset = tf.data.Dataset.from_tensor_slices(dataset)

dataset = dataset.map(split_input_sequence)


BUFFER_SIZE = 1000
BATCH_SIZE = 64

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

for x in dataset.take(1):
    print(x)
    print(np.array(x).shape)
    
print(sum(1 for _ in dataset))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
2022-12-23 15:44:14.233518: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(<tf.Tensor: shape=(64, 114), dtype=int32, numpy=
array([[31, 20, 13, ..., 14, 33, 16],
       [20, 27, 35, ...,  0,  0,  0],
       [24, 27, 34, ...,  0,  0,  0],
       ...,
       [13, 26, 16, ..., 21, 31,  2],
       [21,  2, 35, ..., 20,  2, 24],
       [35, 17, 24, ..., 25, 13, 16]], dtype=int32)>, <tf.Tensor: shape=(64, 114), dtype=int32, numpy=
array([[20, 13, 24, ..., 33, 16, 31],
       [27, 35,  2, ...,  0,  0,  0],
       [27, 34, 17, ...,  0,  0,  0],
       ...,
       [26, 16,  2, ..., 31,  2, 21],
       [ 2, 35, 13, ...,  2, 24, 27],
       [17, 24, 24, ..., 13, 16, 17]], dtype=int32)>)
(2, 64, 114)
156


In [6]:
class QuotesModel(tf.keras.Model):
    def __init__(self, embedding_dim, rnn_units):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        self.rnn = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.rnn.get_initial_state(x)
        x, states = self.rnn(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        return x

embedding_dim = 256
rnn_units = 2048

model = QuotesModel(embedding_dim, rnn_units)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

In [7]:
for x in dataset.take(1):
#     x = np.array(x[0][0])
#     print(x.shape)
#     y = model.predict(x)
#     print(y.shape)
    print(np.array(x).shape)   
    inputs = np.array(x[0])
    print("Inputs: ", inputs.shape)
    x = inputs
    x = model.embedding(x)
    print("Embedding: ", x.shape)
    x = model.rnn(x)
    # (64,127,2048)
    x = model.dense(x[0])
    print("Dense: ", x.shape)
    
    y = model.predict(inputs)
    print("Y: ", y.shape)
    

(2, 64, 114)
Inputs:  (64, 114)
Embedding:  (64, 114, 256)
Dense:  (64, 114, 41)
Y:  (64, 114, 41)


In [8]:
EPOCHS = 10
model.fit(dataset, epochs=EPOCHS)

Epoch 1/10
156/156 [==============================] - 1819s 12s/step - loss: 2.4841 - accuracy: 0.3789
Epoch 2/10
156/156 [==============================] - 1779s 11s/step - loss: 1.7568 - accuracy: 0.4792
Epoch 3/10
156/156 [==============================] - 1769s 11s/step - loss: 1.5605 - accuracy: 0.5338
Epoch 4/10
156/156 [==============================] - 1780s 11s/step - loss: 1.3951 - accuracy: 0.5795
Epoch 5/10
156/156 [==============================] - 1780s 11s/step - loss: 1.2667 - accuracy: 0.6172
Epoch 6/10
156/156 [==============================] - 1780s 11s/step - loss: 1.1714 - accuracy: 0.6447
Epoch 7/10
156/156 [==============================] - 1787s 11s/step - loss: 1.1010 - accuracy: 0.6645
Epoch 8/10
156/156 [==============================] - 1788s 11s/step - loss: 1.0454 - accuracy: 0.6798
Epoch 9/10
156/156 [==============================] - 1764s 11s/step - loss: 0.9966 - accuracy: 0.6936
Epoch 10/10
156/156 [==============================] - 1767s 11s/step - l

In [9]:
for x in dataset.take(1):
    x = np.array(x[0][0])
    y = model.predict(x)
    print(y.shape)
    print(y)
    pred = y[:, -1, :]
    print(pred)

(114, 1, 41)
[[[ 1.8905189  -1.1919647   2.4091408  ... -2.8418636  -2.2466638
   -1.2191489 ]]

 [[-0.81998533 -1.5942986   1.6863627  ... -1.5216067  -1.6356951
   -1.896911  ]]

 [[-3.6063762  -3.0635989  -3.2264922  ... -1.6439639  -3.0954165
   -3.440505  ]]

 ...

 [[ 1.5982934  -4.8537297   3.711016   ... -3.5622747  -4.5389953
   -3.5859728 ]]

 [[-3.6063764  -3.0635989  -3.226492   ... -1.6439639  -3.0954165
   -3.440505  ]]

 [[-3.4479544  -2.6148796  -1.3478332  ... -3.322566   -3.260182
   -2.3942966 ]]]
[[ 1.8905189  -1.1919647   2.4091408  ... -2.8418636  -2.2466638
  -1.2191489 ]
 [-0.81998533 -1.5942986   1.6863627  ... -1.5216067  -1.6356951
  -1.896911  ]
 [-3.6063762  -3.0635989  -3.2264922  ... -1.6439639  -3.0954165
  -3.440505  ]
 ...
 [ 1.5982934  -4.8537297   3.711016   ... -3.5622747  -4.5389953
  -3.5859728 ]
 [-3.6063764  -3.0635989  -3.226492   ... -1.6439639  -3.0954165
  -3.440505  ]
 [-3.4479544  -2.6148796  -1.3478332  ... -3.322566   -3.260182
  -2.3942

In [10]:
class OneStepModel(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars):
        super().__init__()
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars
        
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]

        sparse_mask = tf.SparseTensor(
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        
        predicted_logits = predicted_logits[:, -1, :]
        
        predicted_logits = predicted_logits + self.prediction_mask
        
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        predicted_chars = self.chars_from_ids(predicted_ids)
            
        return predicted_chars, states
    
    
one_step_model = OneStepModel(model, ids_lookup, string_lookup)

states = None

next_char = tf.constant(["i"])

result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)
    
result = tf.strings.join(result)

print(result)



tf.Tensor([b'i only give you a ligar everyone has tanget what i wanted in the universe but thats not a fictionag vilveriescontrull than mincterate with being tround dont read it rather betwelf that think that im diconguops a circumatine lightnyangs alive stooked her that she said women the way you imentife it isnt run something from all the rest of mental intenjectively intentions may anywhere our two of the passive got no voice in the pare you can  ant rather toagh insomental when there would art it colleges back into high the worlds on the heat of an alghostic 1q adage2 zelentlature of your convaislick theyre not any slughttanders shigky braddus and dame wo dont mean  anysone that and not live youdlooking withraid me to called if you want to can lioted them sometimes i can no more than person but sleepingy waiting for as i need it she haz been goodal me which invitch my names at my animension special blem the papsles wize and pug how women are no mats with those servain knowy he said